In [1]:
import os
import json
import jsonschema
import requests

In [4]:
script_path = ''
DATAPATH = os.path.join(script_path,'sample data/')

with open(os.path.join(DATAPATH,'massbank_sample.json'),'r') as massfile:
    massbank_sample = json.load(massfile)

bioschemas_raw = requests.get('https://raw.githubusercontent.com/gtsueng/DDE_bioschemas/main/bioschemas.json')
bioschemas = bioschemas_raw.json()

#### Function to fetch the validations from a DDE-generated schema
def get_validation_from_schema(ddeschema):
    validation_dict = {}
    for eachentry in ddeschema["@graph"]:
        try:
            validation = eachentry["$validation"]
            validation_dict[eachentry["@id"]] = validation
        except:
            continue
    return(validation_dict)



In [5]:
#### Note, the validate function must be run against the validation section in the schema
#### To make things easier, we pull out the validation section in the schema
validation_dict = get_validation_from_schema(bioschemas)
print(validation_dict.keys())
garbagetest = {"name": {"LOL":"This is invalid"}}
#jsonschema.validate(garbagetest,bioschemas["@graph"][18]["$validation"])
jsonschema.validate(massbank_sample[0],validation_dict["bioschemas:MolecularEntity"])

dict_keys(['bioschemas:Taxon', 'bioschemas:BioChemEntity', 'bioschemas:MolecularEntity', 'bioschemas:Gene', 'bioschemas:Dna', 'bioschemas:Rna', 'bioschemas:Protein'])
